In [1]:
import pulp
import pandas as pd
import numpy as np

class PuLP_Solver_Capsule:
    def __init__(self, name="MCM_Optimization", sense='max'):
        """
        初始化优化器
        :param name: 问题名称
        :param sense: 'max' (最大化) 或 'min' (最小化)
        """
        self.sense = pulp.LpMaximize if sense == 'max' else pulp.LpMinimize
        self.prob = pulp.LpProblem(name, self.sense)
        self.vars = {}  # 存储所有变量对象
        self.constraints = {} # 存储约束引用
        self.objective_terms = [] # 存储目标函数项，用于审计
        self.is_solved = False

    def add_variables(self, names, low_bound=0, up_bound=None, cat='Continuous'):
        """
        批量添加变量 (Phase 1 关键步骤)
        :param names: 变量名列表 (如 ['x1', 'x2'] 或 Sku_List)
        :param cat: 'Continuous' (连续), 'Integer' (整数), 'Binary' (0/1)
        """
        # PuLP 的 category 映射
        cat_map = {
            'Continuous': pulp.LpContinuous,
            'Integer': pulp.LpInteger,
            'Binary': pulp.LpBinary
        }
        
        # 创建变量字典
        new_vars = pulp.LpVariable.dicts(
            "Var", 
            names, 
            lowBound=low_bound, 
            upBound=up_bound, 
            cat=cat_map.get(cat, pulp.LpContinuous)
        )
        
        # 更新全局变量库，允许通过原始名称访问
        self.vars.update(new_vars)
        print(f"✅ 已添加 {len(names)} 个变量 (类型: {cat})")
        return new_vars

    def set_weighted_objective(self, weights_dict, objectives_dict, normalize=True):
        """
        设置加权目标函数 (支持多目标归一化)
        :param weights_dict: 权重字典 {'Cost': 0.6, 'Sat': 0.4}
        :param objectives_dict: 目标表达式字典 {'Cost': lpSum(...), 'Sat': lpSum(...)}
        :param normalize: 是否进行最大最小归一化 (简易版，除以预估最大值)
        """
        # 注意：PuLP 不支持非线性，这里假设 objectives_dict 里已经是线性表达式对象
        
        final_obj = 0
        print("🎯 正在构建加权目标函数...")
        
        for name, expr in objectives_dict.items():
            w = weights_dict.get(name, 0)
            
            # 简单的量纲对齐逻辑 (实际比赛建议在外部计算好 scaling factor)
            # 这里仅做演示，不做自动缩放以防黑箱操作
            term = w * expr
            
            # 如果是最小化问题中的“最大化子目标”（如满意度），需要反向
            # 这里假设用户在外部已经处理好了符号（例如 Minimize Cost - Satisfaction）
            
            final_obj += term
            print(f"   -> 添加目标项: {name} (权重: {w})")
            
        self.prob += final_obj
        self.objective_terms = objectives_dict # 记录用于后续分析

    def add_constraint(self, expr, sense, rhs, name):
        """
        添加约束 (Phase 2 审计重点)
        :param expr: 线性表达式 (如 pulp.lpSum(...))
        :param sense: '<=', '>=', '=='
        :param rhs: 右边常数 (Right Hand Side)
        :param name: 约束名称 (必须唯一，用于审计)
        """
        if sense == '<=':
            constraint = (expr <= rhs)
        elif sense == '>=':
            constraint = (expr >= rhs)
        elif sense == '==':
            constraint = (expr == rhs)
        else:
            raise ValueError("Sense must be <=, >=, or ==")
            
        self.prob += constraint, name
        self.constraints[name] = constraint

    def solve(self, solver=None):
        """
        求解模型 (Phase 3 RPG核心)
        """
        print("\n🚀 开始求解...")
        # 默认使用 CBC solver，不仅免费而且够快
        status = self.prob.solve(solver if solver else pulp.PULP_CBC_CMD(msg=0))
        self.status = pulp.LpStatus[status]
        self.is_solved = True
        
        print(f"📊 求解状态: {self.status}")
        if self.status == 'Optimal':
            print(f"💎 目标函数值: {pulp.value(self.prob.objective)}")
        else:
            print("⚠️ 警告: 未找到最优解，请检查约束冲突！")
            
        return self.status

    def export_audit_matrix(self):
        """
        导出约束矩阵用于审计 (Phase 2 核心功能)
        返回: DataFrame (行=约束, 列=变量系数 + RHS)
        """
        # 这是一个解析 PuLP 内部结构的黑科技，用于生成人类可读的表格
        data = []
        
        for name, c in self.prob.constraints.items():
            row = {'_Constraint_Name': name}
            # 解析表达式中的变量系数
            for var, coeff in c.items():
                row[var.name] = coeff
            
            # 获取 RHS (注意 PuLP 内部存储是 expr + constant check 0)
            # 简单处理：直接读取约束的常数项
            row['_RHS'] = -c.constant 
            data.append(row)
            
        df = pd.DataFrame(data).fillna(0)
        return df

    def get_results_df(self):
        """
        获取结果表格 (Phase 5 交付)
        """
        if not self.is_solved:
            return None
        
        data = []
        for v in self.prob.variables():
            data.append({
                'Variable': v.name,
                'Value': v.varValue,
                'Type': v.cat
            })
        return pd.DataFrame(data)

    def get_shadow_prices(self):
        """
        获取影子价格 (Phase 4 灵敏度分析)
        注意：仅对连续变量模型有效，混合整数规划(MIP)通常无影子价格
        """
        data = []
        for name, c in self.prob.constraints.items():
            try:
                # pi 是影子价格，slack 是松弛变量
                data.append({
                    'Constraint': name,
                    'Shadow_Price': c.pi,
                    'Slack': c.slack
                })
            except:
                pass # MIP 问题可能无法获取
        return pd.DataFrame(data)

# ==========================================
# 指挥官备忘录 (Commander Cheat Sheet)
# ==========================================
# 1. 实例化: solver = PuLP_Solver_Capsule(sense='min')
# 2. 变量: vars = solver.add_variables(names_list, cat='Integer')
# 3. 构造表达式: expr = pulp.lpSum([vars[i] * cost[i] for i in names_list])
# 4. 目标: solver.set_weighted_objective({'Cost':1}, {'Cost': expr})
# 5. 约束: solver.add_constraint(expr, '<=', 100, 'Budget_Limit')
# 6. 审计: df_audit = solver.export_audit_matrix() -> 保存为xlsx检查
# 7. 求解: solver.solve()

### 代码与工作流的映射解析

#### 1. `add_variables(..., cat='Integer')`
* **对应工作流：** **Phase 1 逻辑检查**。
* **作用：** 你在 Phase 1 明确要求 AI 检查“人数必须为整数”。如果 AI 写代码时用了默认参数，你在 Jupyter 里看到 `cat='Continuous'` 就能立刻发现并修正。

#### 2. `set_weighted_objective(...)`
* **对应工作流：** **Phase 3 权衡博弈**。
* **作用：** 支持多目标加权。这个函数允许你传入一个权重字典 `weights_dict`。在 Phase 3，当你发现结果不合理时，不需要改动核心逻辑代码，只需要修改传入的字典参数（例如把 `{Cost: 0.5}` 改成 `{Cost: 0.8}`），完美契合“RPG 回环”。

#### 3. `export_audit_matrix()`
* **对应工作流：** **Phase 2 矩阵审计**。
* **作用：** 这是整个类的灵魂。PuLP 的默认输出很难看懂。这个函数会把模型转换成一个 Excel 表格：
    * **行：** 每个约束（如“水资源限制”）。
    * **列：** 每个变量的系数（如“生产A耗水”）。
    * **指挥官操作：** 你只需要打开生成的 Excel，横着扫一眼，就能发现 AI 是不是把系数填错了，或者不等号写反了。这是解决“黑箱建模”的唯一手段。

#### 4. `get_shadow_prices()`
* **对应工作流：** **Phase 4 灵敏度打击**。
* **作用：** 直接生成资源价值分析表。如果某个约束的 `Shadow_Price` 是 0，说明该资源过剩；如果是 1000，说明它是瓶颈。这是写论文“管理建议”章节的核心依据。

---

### 指挥官的使用建议

在比赛中，**不要让 AI 重新写这个类**。
在 **Phase 1（握手）** 时，直接把下面的提示词发给 AI：

> **【系统指令】**
>
> 我已加载 `class PuLP_Solver_Capsule`。
> 请读取 `00_Raw.xlsx`，实例化 Solver，并使用 `add_variables` 和 `add_constraint` 方法构建模型。特别要求：在求解前，调用 `export_audit_matrix()` 并保存为 `02_Matrix_Check.xlsx` 供我审计。

# 🛡️ 比赛标准化交互流程 (SOP) - 基于 PuLP_Solver_Capsule

**前置条件：**
1.  工作目录已包含 `00_Raw.xlsx` (基础数据)。
2.  Notebook 第一个单元格已运行 `class PuLP_Solver_Capsule` 的定义代码。

---

## Phase 0: 环境初始化与数据加载
**场景**：比赛开始，你刚把数据清洗好存入 Excel，准备让 AI 开始建模。
**目的**：防止 AI 幻觉，强制它读取真实数据，并加载封装好的工具类。

> **📋 复制给 AI 的提示词 (Prompt 1)**
>
> ```text
> 【系统指令：初始化环境】
> 1. 我已在环境中定义了 class PuLP_Solver_Capsule。请勿重新编写该类，直接调用即可。
> 2. 请使用 pandas 读取当前目录下的 '00_Raw.xlsx'。
> 3. 展示数据的前 5 行，并简要说明你对数据列含义的理解（这一步是为了确保你读懂了数据字典）。
> ```

---

## Phase 1: 模型构建与矩阵审计 (核心安全阀)
**场景**：建立变量和约束。
**关键动作**：**必须**在求解前生成审计矩阵。这是防止模型逻辑错误的唯一机会。
**对应函数**：`add_variables`, `export_audit_matrix`

> **📋 复制给 AI 的提示词 (Prompt 2)**
>
> ```text
> 【系统指令：构建与审计】
> 请实例化 Solver 并按以下步骤操作：
>
> 1. **变量定义**：调用 .add_variables()。注意：
>    - 如果涉及人员、车辆数量，必须设置 cat='Integer'。
>    - 变量命名需具备可读性（如 x_FactoryA_ProductB）。
>
> 2. **约束构建**：调用 .add_constraint() 添加题目要求的硬性限制（如产能限制、时间限制）。
>
> 3. **核心动作**：不要急着求解！请先调用 .export_audit_matrix()，将结果保存为 '02_Matrix_Check.xlsx'。
>
> 4. **反馈**：告诉我文件已生成，等待我的审计指令。
> ```

**🛑 指挥官动作 (Check)**：
打开 `02_Matrix_Check.xlsx`，检查系数矩阵。
* 确认 $x + y \le 10$ 没有被写成 $x + y \ge 10$。
* 确认消耗系数没有填错位置。

---

## Phase 2: 加权求解与 RPG 回环
**场景**：审计通过，开始求解。如果是多目标优化（既要成本低，又要时间短），在此阶段调整。
**对应函数**：`set_weighted_objective`

> **📋 复制给 AI 的提示词 (Prompt 3)**
>
> ```text
> 【系统指令：设定目标并求解】
> 审计通过。现在进入求解阶段：
>
> 1. 定义目标函数的权重字典。例如：weights = {'Total_Cost': -0.8, 'Satisfaction': 0.2} (注意最小化通常用负号，或根据类内部逻辑调整)。
> 2. 调用 .set_weighted_objective(weights_dict=weights)。
> 3. 执行 .solve()。
> 4. 将求解后的变量结果（Solution）保存为 '03_Solution.xlsx'。
> 5. 打印出最终的目标函数值 (Objective Value) 和求解状态 (Status)。
> ```

**🔄 RPG 回环 (调整)**：
如果结果不合理（例如成本太低但满意度太差），**不要让 AI 改代码**，直接修改 Prompt 中的 `weights` 字典再次运行即可。

---

## Phase 3: 灵敏度打击 (论文加分项)
**场景**：得到最优解后，分析资源的瓶颈，为论文“模型评价”或“管理建议”章节提供数据支持。
**对应函数**：`get_shadow_prices`

> **📋 复制给 AI 的提示词 (Prompt 4)**
>
> ```text
> 【系统指令：灵敏度分析】
> 请执行深度分析：
>
> 1. 调用 .get_shadow_prices()。
> 2. 筛选出 Shadow Price 不为 0 的约束条件（这些是瓶颈资源）。
> 3. 请用自然语言解释这些非零影子价格的含义（例如：“每增加 1 单位水资源，利润可增加 $50”）。
> 4. 将完整的灵敏度分析表保存为 '04_Sensitivity.xlsx'。
> ```

---

## Phase 4: 论文输出 (LaTeX 与 可视化)
**场景**：将代码结果转化为论文素材。

> **📋 复制给 AI 的提示词 (Prompt 5)**
>
> ```text
> 【系统指令：输出论文素材】
> 基于上述模型和结果：
>
> 1. **公式化**：请用标准的 LaTeX 格式写出完整的数学模型（包含索引集合、决策变量、目标函数、约束条件）。确保符号定义清晰。
> 2. **可视化**：读取 '03_Solution.xlsx'，使用 Matplotlib/Seaborn 画一张结果分布图（例如柱状图对比不同工厂的产量），要求配色学术风，并在图中标注关键数据点。
> ```

In [2]:
import pandas as pd

#以此代码生成比赛用的清洗后数据
data = {
    "Route_ID": [f"Route_{i+1:02d}" for i in range(15)], # 唯一主键
    "Origin": ["North_Hub"]*5 + ["South_Hub"]*5 + ["West_Hub"]*5,
    "Destination": ["Shelter_A", "Shelter_B", "Shelter_C", "Shelter_D", "Shelter_E"] * 3,
    "Cost_Per_Unit": [12, 15, 20, 8, 25,  10, 18, 12, 9, 22,  14, 11, 16, 13, 19], # 单箱运输成本
    "Time_Hours":    [2.5, 3.0, 4.0, 1.5, 5.0,  2.0, 3.5, 2.5, 1.8, 4.5,  3.0, 2.2, 3.2, 2.6, 3.8], # 单程耗时
    "Origin_Capacity": [500]*5 + [400]*5 + [300]*5, # 注意：这是冗余列，实际建模需去重
    "Dest_Demand":     [150, 200, 100, 250, 180] * 3 # 注意：这是冗余列，实际建模需去重
}

df = pd.DataFrame(data)

# 模拟“清洗好”的状态：通常清洗后的数据是宽表或长表，这里给最标准的“长表”
# 指挥官注意：Capacity 和 Demand 是附着在 Origin 和 Destination 上的属性
# 但为了方便行读取，它们常被冗余在每一行，建模时需要 Groupby 处理
df.to_excel("00_Cleaned_Data.xlsx", index=False)

print("✅ 战场布置完毕：数据已保存为 00_Cleaned_Data.xlsx")
print("✅ 您的武器 PuLP_Solver_Capsule 此时应已在内存中定义完毕。")

✅ 战场布置完毕：数据已保存为 00_Cleaned_Data.xlsx
✅ 您的武器 PuLP_Solver_Capsule 此时应已在内存中定义完毕。
